# Gossip Semantic Search

In [10]:
# %pip install feedparser
# %pip install -U sentence-transformers
# %pip install tf-keras

In [1]:
import feedparser
import pandas as pd
import json
import numpy as np

from datetime import datetime

## Dataset

## Load the Data

In [39]:
def load_data(output_path):
    data = []
    with open(output_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Erreur de décodage JSON : {e}")

    return pd.DataFrame(data)

output_path_vsd = "../gossip_scrapper/vsd_articles.jl"
output_path_public = "../gossip_scrapper/public_articles.jl"

df_vsd = load_data(output_path_vsd)
df_public = load_data(output_path_public)

print('Number of articles for VSD:', len(df_vsd))
print('Number of articles for Public:', len(df_public))


Number of articles for VSD: 27401
Number of articles for Public: 29569


In [33]:
df = pd.concat([df_vsd, df_public], ignore_index=True)

### Add RSS feed

In [34]:
rss_feeds = [
    "https://www.vsd.fr/rss",
    "https://www.public.fr/rss"
]

articles = []

for feed_url in rss_feeds:
    feed = feedparser.parse(feed_url)
    for entry in feed.entries:
        articles.append({
            'title': entry.title,
            'link': entry.link,
            'published': entry.published,
            'summary': entry.summary
        })

df_rss = pd.DataFrame(articles)

print(f"Number of articles: {len(df_rss)}")
display(df_rss.head())


Number of articles: 100


,title,link,published,summary
0,Un enfant de 2 ans tue sa mère avec l'arme de ...,https://vsd.fr/73487-un-enfant-de-2-ans-tue-sa...,"Mon, 16 Dec 2024 11:15:00 +0000","En manipulant l'arme, l’enfant a appuyé involo..."
1,Les crevettes pailletées chantent à la Seine M...,https://vsd.fr/73466-les-crevettes-pailletees-...,"Mon, 16 Dec 2024 08:51:00 +0000","Imaginée par les artistes Fabrice-Elie Hubert,..."
2,Visite historique du pape François en Corse,https://vsd.fr/73484-visite-historique-du-pape...,"Mon, 16 Dec 2024 06:27:00 +0000","Le pape, qui fêtera ses 88 ans deux jours, a r..."
3,"Tuerie dans le Nord : cinq personnes abattues,...",https://vsd.fr/73477-tuerie-dans-le-nord-cinq-...,"Sun, 15 Dec 2024 18:27:00 +0000",Un jeune homme de 22 ans s'est rendu à la gend...
4,Miss France 2025 : Pourquoi Angélique Angarni-...,https://vsd.fr/73471-miss-france-2025-pourquoi...,"Sun, 15 Dec 2024 16:54:56 +0000","En 2011, Angélique Angarni-Filopon terminait p..."


In [35]:
def convert_date(date_str: str) -> str:
    date_obj = datetime.strptime(date_str, '%a, %d %b %Y %H:%M:%S %z')
    return date_obj.strftime('%Y-%m-%dT%H:%M:%S%z')[:-4] + '00:00'

df_rss = df_rss.rename(columns={
    'link': 'url',
    'published': 'date'
})

df_rss['date'] = df_rss['date'].apply(convert_date)
df = pd.concat([df_rss, df], ignore_index=True)

print(f"Number of articles: {len(df)}")
display(df.head())

Number of articles: 57070


,title,url,date,summary
0,Un enfant de 2 ans tue sa mère avec l'arme de ...,https://vsd.fr/73487-un-enfant-de-2-ans-tue-sa...,2024-12-16T11:15:00+00:00,"En manipulant l'arme, l’enfant a appuyé involo..."
1,Les crevettes pailletées chantent à la Seine M...,https://vsd.fr/73466-les-crevettes-pailletees-...,2024-12-16T08:51:00+00:00,"Imaginée par les artistes Fabrice-Elie Hubert,..."
2,Visite historique du pape François en Corse,https://vsd.fr/73484-visite-historique-du-pape...,2024-12-16T06:27:00+00:00,"Le pape, qui fêtera ses 88 ans deux jours, a r..."
3,"Tuerie dans le Nord : cinq personnes abattues,...",https://vsd.fr/73477-tuerie-dans-le-nord-cinq-...,2024-12-15T18:27:00+00:00,Un jeune homme de 22 ans s'est rendu à la gend...
4,Miss France 2025 : Pourquoi Angélique Angarni-...,https://vsd.fr/73471-miss-france-2025-pourquoi...,2024-12-15T16:54:56+00:00,"En 2011, Angélique Angarni-Filopon terminait p..."


### Data Cleaning

In [36]:
df_cleaned = df.replace("N/A", pd.NA).dropna()
df_cleaned = df_cleaned.drop_duplicates(subset=['url'])

print(f"Number of articles: {len(df_cleaned)}")
display(df_cleaned.head())

df_cleaned.to_csv('../data/articles.csv', index=False)

Number of articles: 42414


,title,url,date,summary
0,Un enfant de 2 ans tue sa mère avec l'arme de ...,https://vsd.fr/73487-un-enfant-de-2-ans-tue-sa...,2024-12-16T11:15:00+00:00,"En manipulant l'arme, l’enfant a appuyé involo..."
1,Les crevettes pailletées chantent à la Seine M...,https://vsd.fr/73466-les-crevettes-pailletees-...,2024-12-16T08:51:00+00:00,"Imaginée par les artistes Fabrice-Elie Hubert,..."
2,Visite historique du pape François en Corse,https://vsd.fr/73484-visite-historique-du-pape...,2024-12-16T06:27:00+00:00,"Le pape, qui fêtera ses 88 ans deux jours, a r..."
3,"Tuerie dans le Nord : cinq personnes abattues,...",https://vsd.fr/73477-tuerie-dans-le-nord-cinq-...,2024-12-15T18:27:00+00:00,Un jeune homme de 22 ans s'est rendu à la gend...
4,Miss France 2025 : Pourquoi Angélique Angarni-...,https://vsd.fr/73471-miss-france-2025-pourquoi...,2024-12-15T16:54:56+00:00,"En 2011, Angélique Angarni-Filopon terminait p..."


## Sentence-transformer model

In [6]:
from sentence_transformers import SentenceTransformer

sentences = df_cleaned['title'].tolist()
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)

np.save('../data/embeddings.npy', embeddings)

d:\2.EPITA\SCIA_Env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
embeddings = np.load('../data/embeddings.npy')

print(embeddings.shape)

(17398, 384)


In [ ]:
def update_encoder(df, model, embeddings):
    sentences = df['title'].tolist()
    new_embeddings = model.encode(sentences)
    return np.concatenate([embeddings, new_embeddings])

    np.save('../data/embeddings.npy', embeddings)

# Semantic Search

### Similarity Test

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

similarité = cosine_similarity([embeddings[0]], [embeddings[10]])
print(f"similarity between two embeddings is : {similarité[0][0]:.4f}")

similarity between two embeddings is : 0.4322


## Get Most Similar Function

In [9]:
from typing import List, Dict, Any

def get_most_similar(sentence:str,
                     embeddings: List[List[float]],
                     model: SentenceTransformer,
                     articles: pd.DataFrame,
                     n:int=5) -> List[Dict[str, Any]]:
    """
    Find the articles most similar to a given sentence using embeddings and cosine similarity.

    Args:
        sentence (str): The query sentence.
        embeddings (List[List[float]]): List of embeddings representing the articles.
        model (SentenceTransformer): The sentence transformer model.
        articles (pd.DataFrame): DataFrame containing the articles with columns ('title', 'url', 'date', 'summary').
        n (int, optional): Number of similar articles to return, defaults to 5.

    Returns:
        List[Dict[str, Any]]: List of dictionaries containing the most similar articles :
            - 'title' (str)
            - 'link' (str)
            - 'date' (str), formatted as 'dd/mm/yy'.
            - 'description
    """

    query_embedding = model.encode(sentence).reshape(1, -1)
    
    similarities = cosine_similarity(np.array(embeddings), query_embedding).flatten()
    
    top_n_indices = similarities.argsort()[-n:][::-1]
    
    most_similar = []
    for idx in top_n_indices:
        article = articles.iloc[idx]
        date = datetime.strptime(article['date'], '%Y-%m-%dT%H:%M:%S%z')
        formatted_date = date.strftime('%d/%m/%y')
        most_similar.append({
            'title': article['title'],
            'url': article['url'], 
            'date': formatted_date,
            'description': article['summary']
        })
    
    return most_similar

In [15]:
sentence = "football"
most_sim = get_most_similar(sentence, embeddings, model, df_cleaned)
for sim in most_sim:
    print(sim)

{'title': 'Sport. Liverpool-Tottenham : les 10 choses à savoir avant la finale - VSD', 'url': 'https://vsd.fr/25317-sport-liverpool-tottenham-les-10-choses-a-savoir-avant-la-finale/', 'date': '01/06/19', 'description': 'La finale de la Ligue des champions 2018-2019 opposera samedi 1er juin, à 21 h, au Wanda Metropolitano de Madrid (l’antre de l’Atlecito) Liverpool à Tottenham.Focus sur deux clubs mythiques de l’Angleterre.'}
{'title': 'SPORT. Les Américaines au sommet - VSD', 'url': 'https://vsd.fr/23946-sport-les-americaines-au-sommet/', 'date': '25/06/19', 'description': 'Elle est «\xa0THE\xa0» star du foot féminin. Et probablement l’une des meilleures joueuses de la planète. Alex Morgan, co-capitaine de l’équipé américaine de soccer, défie les Bleues, vendredi soir, à Paris, en quart de finale de la coupe du monde.'}
{'title': 'L’EURO de football - VSD', 'url': 'https://vsd.fr/1030-l-euro-de-football/', 'date': '14/06/21', 'description': 'Qui pour succéder au Portugal ? Du 11 juin a